## Exercises: Group and aggregate

In [1]:
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset("titanic")

**Use group by and aggregation on the "titanic" dataset to answer the following questions:**

**Q1:** How many males and how many females were onboard titanic?

<details>
<summary>Answer</summary>
<br>
&nbsp;&nbsp;&nbsp;<b>males:</b> 577<br>
&nbsp;&nbsp;&nbsp;<b>females:</b> 314<br>
</details>

In [181]:
sexes = titanic.groupby("sex")

males = sexes["sex"].count().loc["male"]
print(f"males: {males}")

females = sexes["sex"].count().loc["female"]
print(f"females: {females}")

males: 577
females: 314


or:

In [182]:
sexes = titanic.groupby("sex").agg(
    passengers = pd.NamedAgg(column="who", aggfunc="count")
)

sexes.index.name = None
sexes.rename(index={"male": "males", "female": "females"}, inplace=True)
sexes.sort_index(ascending=False, inplace=True)
sexes

,passengers
males,577
females,314


**Q2:** How many males and how many females survived?

<details>
<summary>Answer</summary>
<br>
&nbsp;&nbsp;&nbsp;<b>males:</b> 109<br>
&nbsp;&nbsp;&nbsp;<b>females:</b> 233<br>
</details>

In [79]:
survivors = titanic.groupby(["sex", "alive"]).count()["survived"]

male_survivors = survivors.loc[("male", "yes")]
print(f"males: {male_survivors}")

female_survivors = survivors.loc[("female", "yes")]
print(f"females: {female_survivors}")

males: 109
females: 233


or:

In [120]:
sexes = titanic.groupby("sex").agg(
    survivors = pd.NamedAgg(column="survived", aggfunc="sum")
)

sexes.index.name = None
sexes.rename(index={"male": "males", "female": "females"}, inplace=True)
sexes.sort_index(ascending=False, inplace=True)
sexes

,survivors
males,109
females,233


**Q3:** How many people traveled in each class, and what was the total amount spent on tickets in each class?

<details>
<summary>Answer</summary>
<br>
&nbsp;&nbsp;&nbsp;<b>First:</b> 216 people (£18177)<br>
&nbsp;&nbsp;&nbsp;<b>Second:</b> 184 people (£3802)<br>
&nbsp;&nbsp;&nbsp;<b>Third:</b> 491 people (£6715)<br>
</details>

In [186]:
classes= titanic.groupby("class", observed=True).agg(
    passengers = pd.NamedAgg(column="who", aggfunc="count"),
    total_spent = pd.NamedAgg(column="fare", aggfunc="sum")
)


classes["passengers"] = classes["passengers"].apply(str) + " people"
classes["total_spent"] = "(£" + classes["total_spent"].apply(round).apply(str) + ")"

classes.rename(columns={"total_spent": "(total amount spent on tickets)"}, inplace=True)

# (optional) remove index label "class"
classes.index.name = None

classes

,passengers,(total amount spent on tickets)
First,216 people,(£18177)
Second,184 people,(£3802)
Third,491 people,(£6715)


**Q4:** Where did most people embark, and what town had the highest ratio of survivors?

<details>
<summary>Answer</summary>
<br>
&nbsp;&nbsp;&nbsp;Most people (644) embarked in Southampton.<br>
&nbsp;&nbsp;&nbsp;However,Cherbourg had the highest ratio of survivors (55.4%).
</details>

In [190]:
towns = titanic.groupby("embark_town").agg(
    passengers = pd.NamedAgg(column="survived", aggfunc="count"),
    survivors = pd.NamedAgg(column="survived", aggfunc="sum")
)

towns["survival ratio"] = towns["survivors"] / towns["passengers"]

town_most_passengers = towns["passengers"].idxmax()
town_most_passengers_count = towns["passengers"].loc[town_most_passengers]
print(f"Most people ({town_most_passengers_count}) embarked in {town_most_passengers}.")

town_highest_survival = towns["survival ratio"].idxmax()
town_highest_survival_ratio = towns["survival ratio"].loc[town_highest_survival]
print(f"However, {town_highest_survival} had the highest ratio of survivors ({town_highest_survival_ratio:.1%}).")

Most people (644) embarked in Southampton.
However, Cherbourg had the highest ratio of survivors (55.4%).


The `DataFrame` object used in the cell above:

In [191]:
def ratio_to_percentage(ratio):
    return str(round(ratio * 100, 1)) + "%"

towns["survival ratio"] = towns["survival ratio"].apply(ratio_to_percentage)
towns.drop(columns=["survivors"], inplace=True)

towns

,passengers,survival ratio
embark_town,,
Cherbourg,168,55.4%
Queenstown,77,39.0%
Southampton,644,33.7%


**Q5:** How many family members did survivors travel with on average? And for non-survivors?

<details>
<summary>Answer</summary>
<br>
&nbsp;&nbsp;&nbsp;<b>Survivors:</b> 0.94 family members<br>
&nbsp;&nbsp;&nbsp;<b>Non-Survivors:</b> 0.88 family members<br><br>
&nbsp;&nbsp;&nbsp;<i>* Family members counted as: sibsp + parch</i>
</details>

In [193]:
# columns "sibsp" and "parch" will contain averages:
survivors = titanic.groupby("survived").agg(
    sibsp = pd.NamedAgg(column="sibsp", aggfunc="mean"),
    parch = pd.NamedAgg(column="parch", aggfunc="mean")
)

# sum "sibsp" and "parch" as both count as family members
survivors["family"] = survivors["sibsp"] + survivors["parch"]

survivors.sort_values(by="family", ascending=False, inplace=True)

n_to_family_members = lambda n: str(round(n, 2)) + " family members"
survivors["family"] = survivors["family"].apply(n_to_family_members)

survivors.rename(columns={"family": "average"}, inplace=True)

# columns "sibsp" and "parch" are no longer needed
survivors.drop(columns=["sibsp", "parch"], inplace=True)

survivors.rename(index={0: "Non-Survivors", 1: "Survivors"}, inplace=True)
survivors.index.name = None

survivors

,average
Survivors,0.94 family members
Non-Survivors,0.88 family members


**Use group by and aggregation on the "titanic" dataset to solve the following tasks:**

**T1:** Extract the following data from the dataset:
<!DOCTYPE html>
<html>
<body>

<table style="font-family: Arial, sans-serif; border-collapse: collapse; width: 50%; font-size: 12px;">
  <tr>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;"></th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">who</th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">number_of_passengers</th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">number_of_survivors</th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">ratio_survivors</th>
  </tr>
  <tr>
    <td style="border: none; text-align: right; padding: 8px; color: black;">0</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">woman</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">271</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">205</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">0.756458</td>
  </tr>
  <tr>
    <td style="border: none; text-align: right; padding: 8px; color: black;">1</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">man</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">537</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">88</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">0.163873</td>
  </tr>
  <tr>
    <td style="border: none; text-align: right; padding: 8px; color: black;">2</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">child</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">83</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">49</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">0.590361</td>
  </tr>
</table>

</body>
</html>
<br>
<details>
<summary>Solution</summary>
<pre><code>
titanic.groupby("who").aggregate(
    number_of_passengers=pd.NamedAgg(column="survived", aggfunc="count"),
    number_of_survivors=pd.NamedAgg(column="survived", aggfunc="sum"),
    ratio_survivors=pd.NamedAgg(column="survived", aggfunc="mean"),
).sort_index(ascending=False).reset_index()
</code></pre>
</details>

In [196]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [214]:
whos = titanic.groupby("who").agg(
    number_of_passengers = pd.NamedAgg(column="who", aggfunc="count"),
    number_of_survivors = pd.NamedAgg(column="survived", aggfunc="sum"),
    ratio_survivors = pd.NamedAgg(column="survived", aggfunc="mean")
).sort_values(by="number_of_survivors", ascending=False).reset_index()

whos

,who,number_of_passengers,number_of_survivors,ratio_survivors
0,woman,271,205,0.756458
1,man,537,88,0.163873
2,child,83,49,0.590361


**T2:** Create a custom function *percent()* that can be used to aggregate survivor and get a percentage string:

<!DOCTYPE html>
<html>
<body>

<table style="font-family: Arial, sans-serif; border-collapse: collapse; width: 50%; font-size: 12px;">
  <tr>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;"></th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">who</th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">number_of_passengers</th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">number_of_survivors</th>
    <th style="border: none; text-align: right; padding: 8px; background-color: #f2f2f2; color: black;">percentage_survivors</th>
  </tr>
  <tr>
    <td style="border: none; text-align: right; padding: 8px; color: black;">0</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">woman</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">271</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">205</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">75.6 %</td>
  </tr>
  <tr>
    <td style="border: none; text-align: right; padding: 8px; color: black;">1</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">man</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">537</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">88</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">16.4 %</td>
  </tr>
  <tr>
    <td style="border: none; text-align: right; padding: 8px; color: black;">2</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">child</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">83</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">49</td>
    <td style="border: none; text-align: right; padding: 8px; color: black;">59.0 %</td>
  </tr>
</table>
</body>
</html>

<br>
<details>
<summary>Solution</summary>
<pre><code>
import numpy as np
def percent(col):
    return f"{np.mean(col) * 100:.1f} %"
</code></pre>
</details>


In [215]:
percent = lambda n: f"{n:.1%}"

whos = titanic.groupby("who").agg(
    number_of_passengers = pd.NamedAgg(column="who", aggfunc="count"),
    number_of_survivors = pd.NamedAgg(column="survived", aggfunc="sum"),
    ratio_survivors = pd.NamedAgg(column="survived", aggfunc="mean")
).sort_values(by="number_of_survivors", ascending=False).reset_index()

whos["ratio_survivors"] = whos["ratio_survivors"].apply(percent)

whos

,who,number_of_passengers,number_of_survivors,ratio_survivors
0,woman,271,205,75.6%
1,man,537,88,16.4%
2,child,83,49,59.0%
